In [1]:
import os

# Tên thư mục gốc
root_dir = "BrainTumorDiagnosisUsingMRI"

# Danh sách các thư mục con cần tạo (theo PDF)
folders = [
    "data/kaggle",
    "src/preprocessing",
    "src/dataloader",
    "src/models",
    "src/training",
    "src/evaluation",
    "src/inference",
    "src/utils",
    "notebooks",
    "api",
    "app"
]

# Danh sách các file cần tạo (để giữ chỗ)
files = [
    "api/fastapi_app.py",
    "app/streamlit_app.py",
    "notebooks/EDA.ipynb",
    "notebooks/Training_ResNet.ipynb",
    "notebooks/Training_EfficientNet.ipynb",
    "notebooks/GradCAM.ipynb",
    "README.md",
    ".gitignore"
]

# 1. Tạo thư mục
print(f"Đang tạo cấu trúc thư mục tại: {root_dir}/")
for folder in folders:
    path = os.path.join(root_dir, folder)
    os.makedirs(path, exist_ok=True)
    print(f"✅ Đã tạo: {path}")

# 2. Tạo file rỗng
for file in files:
    path = os.path.join(root_dir, file)
    with open(path, 'w') as f:
        pass # Tạo file rỗng
    print(f"📄 Đã tạo file: {path}")

print("\n🎉 Cấu trúc dự án đã sẵn sàng!")

Đang tạo cấu trúc thư mục tại: BrainTumorDiagnosisUsingMRI/
✅ Đã tạo: BrainTumorDiagnosisUsingMRI/data/kaggle
✅ Đã tạo: BrainTumorDiagnosisUsingMRI/src/preprocessing
✅ Đã tạo: BrainTumorDiagnosisUsingMRI/src/dataloader
✅ Đã tạo: BrainTumorDiagnosisUsingMRI/src/models
✅ Đã tạo: BrainTumorDiagnosisUsingMRI/src/training
✅ Đã tạo: BrainTumorDiagnosisUsingMRI/src/evaluation
✅ Đã tạo: BrainTumorDiagnosisUsingMRI/src/inference
✅ Đã tạo: BrainTumorDiagnosisUsingMRI/src/utils
✅ Đã tạo: BrainTumorDiagnosisUsingMRI/notebooks
✅ Đã tạo: BrainTumorDiagnosisUsingMRI/api
✅ Đã tạo: BrainTumorDiagnosisUsingMRI/app
📄 Đã tạo file: BrainTumorDiagnosisUsingMRI/api/fastapi_app.py
📄 Đã tạo file: BrainTumorDiagnosisUsingMRI/app/streamlit_app.py
📄 Đã tạo file: BrainTumorDiagnosisUsingMRI/notebooks/EDA.ipynb
📄 Đã tạo file: BrainTumorDiagnosisUsingMRI/notebooks/Training_ResNet.ipynb
📄 Đã tạo file: BrainTumorDiagnosisUsingMRI/notebooks/Training_EfficientNet.ipynb
📄 Đã tạo file: BrainTumorDiagnosisUsingMRI/notebook

In [2]:
import zipfile
import shutil

# Giả sử bạn đã upload file dataset.zip lên Colab
zip_path = 'dataset.zip'
extract_path = os.path.join(root_dir, 'data/kaggle')

if os.path.exists(zip_path):
    print(f"Đang giải nén {zip_path} vào {extract_path}...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("✅ Giải nén thành công!")

    # Kiểm tra xem giải nén xong nó có tạo thêm thư mục con không
    # Ví dụ: data/kaggle/dataset/Glioma... hay là data/kaggle/Glioma...
    print("Cấu trúc trong data/kaggle:")
    print(os.listdir(extract_path))
else:
    print("⚠️ Lỗi: Không tìm thấy file 'dataset.zip'. Hãy upload lên Colab trước!")

Đang giải nén dataset.zip vào BrainTumorDiagnosisUsingMRI/data/kaggle...
✅ Giải nén thành công!
Cấu trúc trong data/kaggle:
['Testing', 'Training']


In [3]:
!pip install split-folders opencv-python-headless

import splitfolders

# Đường dẫn đầu vào (Nơi vừa giải nén dataset)
# LƯU Ý: Bạn cần kiểm tra xem sau khi giải nén, các folder Glioma, Meningioma... nằm ở đâu.
# Nếu nó nằm trực tiếp trong data/kaggle thì dùng dòng dưới:
input_folder = os.path.join(root_dir, 'data/kaggle')

# Nếu giải nén ra nó lại nằm trong 1 thư mục con nữa (ví dụ data/kaggle/BrainTumor/...)
# thì bạn phải sửa đường dẫn input_folder cho đúng.

# Đường dẫn đầu ra (Nơi chứa dữ liệu đã chia train/val/test)
output_folder = os.path.join(root_dir, 'data/processed')

print(f"Đang chia dữ liệu từ {input_folder} sang {output_folder}...")

# Chia dữ liệu
try:
    splitfolders.ratio(input_folder, output=output_folder,
                       seed=42, ratio=(.7, .15, .15),
                       group_prefix=None)
    print("✅ Đã chia dữ liệu thành công!")
except Exception as e:
    print(f"❌ Có lỗi xảy ra (thường do đường dẫn input chưa đúng cấp thư mục): {e}")

Đang chia dữ liệu từ BrainTumorDiagnosisUsingMRI/data/kaggle sang BrainTumorDiagnosisUsingMRI/data/processed...


Copying files: 0 files [00:00, ? files/s]

✅ Đã chia dữ liệu thành công!


In [6]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Hàm khử nhiễu (đặt trong src/preprocessing nếu muốn, nhưng ở đây ta chạy trực tiếp)
def remove_noise(image):
    img_uint8 = np.array(image, dtype=np.uint8)
    denoised = cv2.medianBlur(img_uint8, 3)
    return denoised.astype('float32')

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Cấu hình Generator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=remove_noise,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True,
    zoom_range=0.1
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=remove_noise
)

# Trỏ vào thư mục processed vừa tạo ở Bước 1
train_dir = os.path.join(output_folder, 'train')
val_dir = os.path.join(output_folder, 'val')
test_dir = os.path.join(output_folder, 'test')

print("Đang khởi tạo Data Generators...")
train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical'
)

val_generator = test_datagen.flow_from_directory(
    val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False
)

Đang khởi tạo Data Generators...
Found 3996 images belonging to 4 classes.
Found 855 images belonging to 4 classes.
Found 861 images belonging to 4 classes.


In [7]:
from tensorflow.keras import layers, models, Input
from tensorflow.keras.applications import ConvNeXtTiny

def build_hybrid_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    # 1. Backbone: ConvNeXtTiny
    backbone = ConvNeXtTiny(include_top=False, weights='imagenet', input_tensor=inputs)
    backbone.trainable = False
    x = backbone.output

    # 2. Flatten -> Tokens
    x = layers.Reshape((-1, x.shape[-1]))(x)

    # 3. Transformer Encoder
    embed_dim = 768
    num_heads = 8
    ff_dim = 1024

    x = layers.LayerNormalization(epsilon=1e-6)(x)
    attention_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(x, x)
    x = layers.Add()([x, attention_output])
    x = layers.LayerNormalization(epsilon=1e-6)(x)

    ffn = models.Sequential([
        layers.Dense(ff_dim, activation="relu"),
        layers.Dense(embed_dim),
    ])
    ffn_output = ffn(x)
    x = layers.Add()([x, ffn_output])

    # 4. Classifier
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs=inputs, outputs=outputs, name="ConvNeXt_Transformer_Hybrid")
    return model

# Xây dựng mô hình
model = build_hybrid_model(input_shape=(224, 224, 3), num_classes=4)
model.summary()

# Lưu mô hình vào thư mục models trong cấu trúc dự án
model_save_path = os.path.join(root_dir, 'src/models/untrained_hybrid_model.h5')
model.save(model_save_path)
print(f"✅ Đã lưu kiến trúc mô hình tại: {model_save_path}")

111650432/111650432 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "ConvNeXt_Transformer_Hybrid"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_pres… │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stem  │ (None, 56, 56,    │      4,896 │ convnext_tiny_pr… │
│ (Sequential)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │      4,800 │ convnext_tiny_st… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │        192 │ convnext_tiny_st… │
│ (LayerNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │     37,248 │ convnext_tiny_st… │
│ (Dense)             │ 384)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │          0 │ convnext_tiny_st… │
│ (Activation)        │ 384)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │     36,960 │ convnext_tiny_st… │
│ (Dense)             │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │         96 │ convnext_tiny_st… │
│ (LayerScale)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │          0 │ convnext_tiny_st… │
│ (Activation)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 56, 56,    │          0 │ convnext_tiny_st… │
│                     │ 96)               │            │ convnext_tiny_st… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │      4,800 │ add[0][0]         │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │        192 │ convnext_tiny_st… │
│ (LayerNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │     37,248 │ convnext_tiny_st… │
│ (Dense)             │ 384)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │          0 │ convnext_tiny_st… │
│ (Activation)        │ 384)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │     36,960 │ convnext_tiny_st… │
│ (Dense)             │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │         96 │ convnext_tiny_st

 Total params: 48,294,500 (184.23 MB)

 Trainable params: 20,474,372 (78.10 MB)

 Non-trainable params: 27,820,128 (106.13 MB)

✅ Đã lưu kiến trúc mô hình tại: BrainTumorDiagnosisUsingMRI/src/models/untrained_hybrid_model.h5


In [8]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import AdamW # AdamW có sẵn trong TF 2.11+

# 1. Định nghĩa các Callbacks (Trợ lý huấn luyện)
# Lưu model vào đúng thư mục src/models
checkpoint_path = os.path.join(root_dir, 'src/models/best_hybrid_model.keras')

checkpoint = ModelCheckpoint(
    checkpoint_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=5, # Theo PDF
    restore_best_weights=True,
    verbose=1
)

# Giảm tốc độ học nếu không cải thiện
reduce_lr = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.2,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

# 2. Compile Mô hình
# Sử dụng AdamW như yêu cầu PDF
# weight_decay là tham số đặc trưng của AdamW giúp chống overfitting
optimizer = AdamW(learning_rate=0.001, weight_decay=1e-4)

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("🚀 Bắt đầu quá trình huấn luyện Hybrid Model...")

# 3. Bắt đầu Training
# Chạy 30 epochs (nằm trong khoảng 20-40 của PDF)
history = model.fit(
    train_generator,
    epochs=30,
    validation_data=val_generator,
    callbacks=[checkpoint, early_stopping, reduce_lr]
)

print("✅ Huấn luyện hoàn tất!")

🚀 Bắt đầu quá trình huấn luyện Hybrid Model...


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 587ms/step - accuracy: 0.3539 - loss: 4.3179
Epoch 1: val_accuracy improved from -inf to 0.66433, saving model to BrainTumorDiagnosisUsingMRI/src/models/best_hybrid_model.keras
125/125 ━━━━━━━━━━━━━━━━━━━━ 117s 753ms/step - accuracy: 0.3545 - loss: 4.3016 - val_accuracy: 0.6643 - val_loss: 0.7896 - learning_rate: 0.0010
Epoch 2/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 503ms/step - accuracy: 0.5751 - loss: 1.0166
Epoch 2: val_accuracy improved from 0.66433 to 0.74971, saving model to BrainTumorDiagnosisUsingMRI/src/models/best_hybrid_model.keras
125/125 ━━━━━━━━━━━━━━━━━━━━ 74s 588ms/step - accuracy: 0.5752 - loss: 1.0165 - val_accuracy: 0.7497 - val_loss: 0.6454 - learning_rate: 0.0010
Epoch 3/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 498ms/step - accuracy: 0.6184 - loss: 0.8865
Epoch 3: val_accuracy did not improve from 0.74971
125/125 ━━━━━━━━━━━━━━━━━━━━ 66s 527ms/step - accuracy: 0.6186 - loss: 0.8862 - val_accuracy: 0.7357 - val_loss: 0.6652 - learnin

In [9]:
# 1. Tải lại mô hình tốt nhất (trạng thái ở Epoch 14 - 84.56%)
model_path = os.path.join(root_dir, 'src/models/best_hybrid_model.keras')
model.load_weights(model_path)
print("Đã tải lại weights tốt nhất từ Epoch 14.")

# 2. Mở khóa Backbone (Unfreeze)
# Lúc nãy ta để backbone.trainable = False. Giờ ta cho phép nó học để tinh chỉnh đặc trưng.
# Vì ConvNeXt nằm bên trong model, ta set toàn bộ model là trainable
model.trainable = True

# Tuy nhiên, để tránh làm hỏng các weights đã học, ta dùng Learning Rate CỰC NHỎ
# Nhỏ hơn 10-100 lần so với lúc đầu.

# 3. Cấu hình lại Callbacks (Nới lỏng Patience)
checkpoint_finetune = ModelCheckpoint(
    os.path.join(root_dir, 'src/models/best_hybrid_model_finetuned.keras'), # Lưu ra file mới
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

# Tăng patience lên 10 để nó có thời gian học
early_stopping_finetune = EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

reduce_lr_finetune = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.2,
    patience=3,
    min_lr=1e-7, # Cho phép giảm sâu hơn nữa
    verbose=1
)

# 4. Compile lại với LR nhỏ (1e-5)
# Lưu ý: Phải compile lại sau khi thay đổi trainable
optimizer_finetune = AdamW(learning_rate=1e-5, weight_decay=1e-5)

model.compile(
    optimizer=optimizer_finetune,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("🚀 Bắt đầu Fine-tuning (Unfreezing Backbone)...")
print("Mục tiêu: Phá vỡ giới hạn 84.5%")

# 5. Train tiếp 20 epochs nữa
history_finetune = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[checkpoint_finetune, early_stopping_finetune, reduce_lr_finetune]
)

Đã tải lại weights tốt nhất từ Epoch 14.
🚀 Bắt đầu Fine-tuning (Unfreezing Backbone)...
Mục tiêu: Phá vỡ giới hạn 84.5%
Epoch 1/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 548ms/step - accuracy: 0.7744 - loss: 0.5270
Epoch 1: val_accuracy improved from -inf to 0.83275, saving model to BrainTumorDiagnosisUsingMRI/src/models/best_hybrid_model_finetuned.keras
125/125 ━━━━━━━━━━━━━━━━━━━━ 105s 700ms/step - accuracy: 0.7744 - loss: 0.5270 - val_accuracy: 0.8327 - val_loss: 0.4292 - learning_rate: 1.0000e-05
Epoch 2/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 497ms/step - accuracy: 0.7977 - loss: 0.5035
Epoch 2: val_accuracy did not improve from 0.83275
125/125 ━━━━━━━━━━━━━━━━━━━━ 66s 526ms/step - accuracy: 0.7977 - loss: 0.5035 - val_accuracy: 0.8316 - val_loss: 0.4272 - learning_rate: 1.0000e-05
Epoch 3/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 495ms/step - accuracy: 0.7976 - loss: 0.5007
Epoch 3: val_accuracy did not improve from 0.83275
125/125 ━━━━━━━━━━━━━━━━━━━━ 66s 525ms/step - accuracy: 0.7975 - loss: 0.50

In [10]:
# Tải lại mô hình tốt nhất (file finetuned)
final_model_path = os.path.join(root_dir, 'src/models/best_hybrid_model_finetuned.keras')
# Nếu file finetuned không tồn tại (do lỗi lưu), dùng file best_hybrid_model.keras cũ
if not os.path.exists(final_model_path):
    final_model_path = os.path.join(root_dir, 'src/models/best_hybrid_model.keras')

model.load_weights(final_model_path)
print(f"Đang đánh giá mô hình: {final_model_path}")

# Đánh giá trên tập Test
test_loss, test_acc = model.evaluate(test_generator)
print(f"\n>>> KẾT QUẢ CUỐI CÙNG TRÊN TẬP TEST: {test_acc * 100:.2f}% <<<")

# Vẽ lại Confusion Matrix để xem nó sai ở đâu
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

test_generator.reset()
Y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(Y_pred, axis=1)
y_true = test_generator.classes
class_names = list(test_generator.class_indices.keys())

print(classification_report(y_true, y_pred_classes, target_names=class_names, digits=4))

Đang đánh giá mô hình: BrainTumorDiagnosisUsingMRI/src/models/best_hybrid_model_finetuned.keras
27/27 ━━━━━━━━━━━━━━━━━━━━ 12s 421ms/step - accuracy: 0.7771 - loss: 0.5293

>>> KẾT QUẢ CUỐI CÙNG TRÊN TẬP TEST: 83.39% <<<
27/27 ━━━━━━━━━━━━━━━━━━━━ 14s 320ms/step
              precision    recall  f1-score   support

      glioma     0.7737    0.7387    0.7558       199
  meningioma     0.7670    0.6683    0.7143       202
     notumor     0.9412    0.9333    0.9372       240
   pituitary     0.8249    0.9636    0.8889       220

    accuracy                         0.8339       861
   macro avg     0.8267    0.8260    0.8240       861
weighted avg     0.8319    0.8339    0.8306       861



In [12]:
import os

gitignore_content = """
# Bỏ qua thư mục dữ liệu
data/
raw_data/
dataset.zip

# Bỏ qua các file cache của python
__pycache__/
*.pyc
.ipynb_checkpoints/

# Giữ lại thư mục src và models, nhưng bỏ qua các file tạm
"""

# Ghi vào thư mục gốc dự án
with open('BrainTumorDiagnosisUsingMRI/.gitignore', 'w') as f:
    f.write(gitignore_content)

print("✅ Đã tạo file .gitignore để loại bỏ dữ liệu nặng.")

✅ Đã tạo file .gitignore để loại bỏ dữ liệu nặng.
